In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn import pipeline,preprocessing,feature_extraction,metrics

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('russian')

from sklearn.base import BaseEstimator 
from sklearn.base import TransformerMixin

from sklearn.pipeline  import FeatureUnion
from sklearn.pipeline  import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer as TfIdf

from sklearn.model_selection import ShuffleSplit
import xgboost as xgb


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")

In [3]:
def preprocess(dfOld: pd.DataFrame) -> pd.DataFrame:
    df = dfOld.copy()
    
    df['description'] = dfOld['description'].replace(r"[.,\/#!$%\^&\*;:{}=\-_`~()]", " ")
    df['city'] = dfOld['city'].replace(r"[.,\/#!$%\^&\*;:{}=\-_`~()]", " ")
    
    df['description_len'] = df['description'].fillna('').map(lambda x: len(str(x))).astype(np.float16) #Length
    df['description_wc'] = df['description'].fillna('').map(lambda x: len(str(x).split(' '))).astype(np.float16) #Word Count
    df['title_len'] = df['title'].fillna('').map(lambda x: len(str(x))).astype(np.float16) #Lenth
    df['title_wc'] = df['title'].fillna('').map(lambda x: len(str(x).split(' '))).astype(np.float16) #Word Count
    
    df['description'] = (df['title'].fillna('') + ' ' + df['description'].fillna('') + ' ' + df['city'])
    df['description'] = df['description'].str.lower().replace(r"[^[:alpha:]]", " ")
    df['description'] = df['description'].str.replace(r"\\s+", " ")
    
    df['param'] = (df['param_1'].fillna('') + ' ' + df['param_2'].fillna('') + ' ' + df['param_3'].fillna(''))
        
    df['image'] = df['image'].fillna('').map(lambda x: 1 if len(str(x))>0 else 0)
    df['price'] = np.log1p(df['price'].fillna(0))
    df['image_top_1'] = np.log1p(df['image_top_1'].fillna(0))
    df['wday'] = pd.to_datetime(df['activation_date']).dt.dayofweek
    df['day'] = pd.to_datetime(df['activation_date']).dt.day
    df['week'] = pd.to_datetime(df['activation_date']).dt.week
    
    ex_col = ['item_id', 'user_id', 'deal_probability', 'title', 'param_1', 'param_2', 'param_3', 'activation_date', 'city']
    col = [c for c in df.columns if c not in ex_col]
    return df[col]

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        #assert isinstance(X, pd.DataFrame)
        return X[self.key]
    
class FeatureInfo(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        #print('FeatureInfo ' + str(type(X)))
        return self
    def transform(self, X):
        print('FeatureInfo ' + str(X.shape) + str(type(X)))
        return X
    
class ArrayCaster(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        #print(data.shape)
        #print(np.transpose(np.matrix(data)).shape)
        return np.transpose(np.matrix(data))

    
class LabelEncoders(BaseEstimator, TransformerMixin):
    def __init__(self, id):
        self.mappings = {}
        self.id = id
    def fit(self, X, y=None):
        name = X.name
        print("LabelEncoders " + name + self.id)
        if (name not in self.mappings):
            self.mappings[name] = preprocessing.LabelEncoder()
        self.mappings[name].fit(X)
        return self
    def transform(self, X):
        result = self.mappings[X.name].transform(X)
        print("LabelEncoders" + str(result.shape))
        return result
    
def on_field(key:str, *transformers) -> Pipeline:
    additional_steps = [ ("step" + str(ii), transformers[ii]) for ii in range(len(transformers)) ]
    steps =  [('selection', FeatureSelector(key=key)),  ('info', FeatureInfo()) ] + additional_steps
    print(steps)
    return Pipeline(steps)

def on_multiple_fields(keys, *transformers) -> Pipeline:
    return FeatureUnion(
        transformer_list=[
            ("pipeline" + str(keys[ii]), on_field(keys[ii], *transformers)) for ii in range(len(keys))
        ]
    )

<h1>Big data experiment</h1>

In [4]:
vectorizer = pipeline.make_pipeline(
                pipeline.make_union(
                    on_field('description', TfIdf(max_features=3500, stop_words=sw, token_pattern='\w+', norm='l2', min_df=3, max_df=0.3, sublinear_tf=True, ngram_range=(1, 3))),
                    on_field('param', TfIdf(max_features=3500, stop_words=sw, token_pattern='\w+', norm='l2',min_df=3, max_df=0.3, sublinear_tf=True, ngram_range=(1, 3))),
                    on_multiple_fields(['region', 'parent_category_name', 'category_name', 'user_type'], LabelEncoders(id="same"), ArrayCaster()),
                    on_multiple_fields(['price', 'item_seq_number', 'image', 'image_top_1', 'description_len', 'description_wc', 'title_len', 'title_wc', 'wday', 'day', 'week'], ArrayCaster())
                ),
                #RandomForestRegressor()
)

# shuffle
ss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=37)
train_index, test_index = next(ss.split(train_df))

train_df_prep = preprocess(train_df)
train_df_Y = train_df['deal_probability']

train_X = train_df_prep.iloc[train_index]
train_Y = train_df['deal_probability'][train_index]

test_X = train_df_prep.iloc[test_index]
test_Y = train_df['deal_probability'][test_index]

[('selection', FeatureSelector(key='description')), ('info', FeatureInfo()), ('step0', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.3, max_features=3500, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=True, token_pattern='\\w+',
        tokenizer=None, use_idf=True, vocabulary=None))]
[('selection', FeatureSelector(key='param')), ('info', FeatureInfo()), ('step0', TfidfVector

In [5]:
vectorizer.fit(train_X)

FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders regionsame
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders parent_category_namesame
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders category_namesame
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders user_typesame
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<cla

Pipeline(memory=None,
     steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(memory=None,
     steps=[('selection', FeatureSelector(key='description')), ('info', FeatureInfo()), ('step0', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
     ...='week')), ('info', FeatureInfo()), ('step0', ArrayCaster())]))],
       transformer_weights=None))],
       transformer_weights=None))])

<h2>Test with randomforestregressor</h2>

In [6]:
train_X_t = vectorizer.transform(train_X)
test_X_t = vectorizer.transform(test_X)

clf = RandomForestRegressor(n_jobs=16)
clf.fit(train_X_t, train_Y)

test_pred = clf.predict(test_X_t)

from sklearn.metrics import mean_squared_error as mse
error = mse(test_Y, test_pred)
print(error)

FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
LabelEncoders(1202739,)
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pandas.core.series.Series'>
FeatureInfo (1202739,)<class 'pand

KeyboardInterrupt: 

<h2>Test with xgboost </h2>

In [8]:
from xgboost import XGBRegressor

xgbclf = XGBRegressor(n_jobs=16, silent=False)
xgbclf.fit(train_X_t, train_Y)



NameError: name 'clf2' is not defined

In [9]:
test_pred_xgb = xgbclf.predict(test_X_t)

from sklearn.metrics import mean_squared_error as mse
error = mse(test_Y, test_pred_xgb)
print(error)

0.05549476162360624


<h2>Test with xgboost </h2>


In [36]:
import scipy.sparse as sp

xgbclf2 = XGBRegressor(n_jobs=16, silent=False)
all_X =  sp.vstack((train_X_t, test_X_t))
all_Y = sp.hstack((train_Y, test_Y)).T
all_X.shape

(1503424, 7015)

In [37]:
print(all_Y.shape)
xgbclf2.fit(all_X, all_Y)

(1503424, 1)


MemoryError: 

In [38]:
test_df_prep = preprocess(test_df)
test_df_prep_t = vectorizer.transform(test_df_prep)
test_df_prep_t_pred = xgbclf.predict(test_df_prep_t)

df_submission = pd.DataFrame()
df_submission['item_id'] = test_df['item_id']
df_submission['deal_probability'] = pd.Series(test_df_prep_t_pred)

df_submission.to_csv('submission.csv', index=False, index_label=False)


FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
LabelEncoders(508438,)
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
LabelEncoders(508438,)
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
LabelEncoders(508438,)
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
LabelEncoders(508438,)
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Series'>
FeatureInfo (508438,)<class 'pandas.core.series.Serie

In [41]:
test_df_prep_t_pred_series = pd.Series(test_df_prep_t_pred)
minn = test_df_prep_t_pred_series.min()
maxx = test_df_prep_t_pred_series.max()

In [42]:
test_df_prep_t_pred_series=test_df_prep_t_pred_series.apply(lambda x: (x-minn)/(maxx-minn))

In [43]:
test_df_prep_t_pred_series.describe()

count    508438.000000
mean          0.258402
std           0.136093
min           0.000000
25%           0.140774
50%           0.240963
75%           0.323774
max           1.000000
dtype: float64

In [44]:
df_submission = pd.DataFrame()
df_submission['item_id'] = test_df['item_id']
df_submission['deal_probability'] = test_df_prep_t_pred_series

df_submission.to_csv('submission.csv', index=False, index_label=False)


In [45]:
test_df_prep_t_pred_series.describe()

count    508438.000000
mean          0.258402
std           0.136093
min           0.000000
25%           0.140774
50%           0.240963
75%           0.323774
max           1.000000
dtype: float64

In [46]:
pd.Series(test_df_prep_t_pred).describe()

count    508438.000000
mean          0.146537
std           0.095903
min          -0.035600
25%           0.063625
50%           0.134243
75%           0.192612
max           0.669253
dtype: float64